# **Data Analysis of Sales of Supermarket Using SQL**

***Use Cases :***
* Calculate total revenue per branch.
* Identify the branch with the highest average sales.
* Determine total sales revenue by month.
* Find peak sales hours across branches.
* Count how many customers are members versus normal customers.
* Analyze the spending differences between male and female customers.
* Identify the most popular payment method among customers.
* List top 5 product lines by total sales revenue.
* Calculate the average quantity sold per product line.
* Find the product line with the highest gross margin.
* Discover sales trends by weekdays versus weekends.
* Identify if particular product lines sell better during certain times of the day (morning vs. evening).

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/supermarket-sales/supermarket_sales - Sheet1.csv


In [2]:
csv_file_path = '/kaggle/input/supermarket-sales/supermarket_sales - Sheet1.csv'
df = pd.read_csv(csv_file_path)
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [3]:
# Create a SQLite connection (in-memory database)
conn = sqlite3.connect(':memory:')

# Load DataFrame into SQLite as a table named 'data_table'
df.to_sql('sales_table', conn, index=False, if_exists='replace')

1000

In [4]:
query = '''
SELECT *
FROM sales_table
LIMIT 5;
'''

result = pd.read_sql_query(query, conn)
print(result)

    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total       Date  \
0       Health and beauty       74.69         7  26.1415  548.9715   1/5/2019   
1  Electronic accessories       15.28         5   3.8200   80.2200   3/8/2019   
2      Home and lifestyle       46.33         7  16.2155  340.5255   3/3/2019   
3       Health and beauty       58.22         8  23.2880  489.0480  1/27/2019   
4       Sports and travel       86.31         7  30.2085  634.3785   2/8/2019   

    Time      Payment    cogs  gross margin percentage  gross income  Rating  
0  13:08      Ewallet  522.83                 4.761905       26.1415     9.1  
1  10:

In [5]:
#get all brances
query = '''
SELECT Count("Invoice ID") AS INVOICE_ID,Branch 
FROM sales_table
GROUP BY Branch;
'''

result = pd.read_sql_query(query, conn)
print(result)

   INVOICE_ID Branch
0         340      A
1         332      B
2         328      C


In [6]:
#Calculate total revenue per branch.
query = '''
SELECT Branch, SUM("Total") AS REVENUE
FROM sales_table
GROUP BY Branch;
'''
result = pd.read_sql_query(query, conn)
print(result)

  Branch      REVENUE
0      A  106200.3705
1      B  106197.6720
2      C  110568.7065


In [7]:
#Branch with the highest average sales.
query = '''
WITH sales_revenue AS (
    SELECT 
    SUM("Total") AS REVENUE,
    COUNT("Invoice Id") AS SALES,
    City as city_branch
    FROM sales_table
    GROUP BY city_branch
)


SELECT 
ROUND(REVENUE / SALES) revenue_per_sale,
DENSE_RANK() OVER (ORDER BY ROUND(REVENUE / SALES) DESC) as sales_rank ,
city_branch
FROM 
sales_revenue;
'''
result = pd.read_sql_query(query, conn)
print(result)

   revenue_per_sale  sales_rank city_branch
0             337.0           1   Naypyitaw
1             320.0           2    Mandalay
2             312.0           3      Yangon


In [8]:
#Determine total sales revenue by month.
query = '''
WITH sales_revenue_month AS (
    SELECT 
    City,
    CAST(substr(DATE, 1, instr(DATE, '/') - 1) AS INTEGER) AS Month_A,
    COUNT("Invoice ID") AS Count_of_Sales,
    SUM("Total") AS revenue
    FROM sales_table
    GROUP BY City, Month_A
)


SELECT 
*
FROM 
sales_revenue_month;
'''

result = pd.read_sql_query(query, conn)
print(result)

        City  Month_A  Count_of_Sales     revenue
0   Mandalay        1             111  37176.0585
1   Mandalay        2             109  34424.2710
2   Mandalay        3             112  34597.3425
3  Naypyitaw        1             122  40434.6810
4  Naypyitaw        2             100  32934.9825
5  Naypyitaw        3             106  37199.0430
6     Yangon        1             119  38681.1285
7     Yangon        2              94  29860.1205
8     Yangon        3             127  37659.1215


In [9]:
#Peak sales hours across branche
query = '''
WITH HourlySales AS (
    SELECT
        City,
        CAST(substr(Time, 1, 2) AS INTEGER) AS Sales_Hour,
        COUNT("Invoice ID") AS Number_of_Sales
        FROM sales_table
    GROUP BY City, Sales_Hour
),
RankedSales AS (
    SELECT
        City,
        Sales_Hour,
        Number_of_Sales,
        DENSE_RANK() OVER (PARTITION By City ORDER BY Number_of_Sales DESC) AS HOUR_RATE
        FROM
    HourlySales
)
SELECT
City,
Sales_Hour,
Number_of_Sales
FROM 
RankedSales
WHERE HOUR_RATE = 1
;
'''

result = pd.read_sql_query(query, conn)
print(result)

        City  Sales_Hour  Number_of_Sales
0   Mandalay          19               50
1  Naypyitaw          10               37
2     Yangon          10               38


In [10]:
#Count how many customers are members versus normal customers.
query = '''
   SELECT
       COUNT("Invoice ID") AS SALES_COUNT,
       "Customer Type" AS Customer_Type
   FROM
   sales_table
   GROUP By Customer_Type;
'''

result = pd.read_sql_query(query, conn)
print(result)

   SALES_COUNT Customer_Type
0          501        Member
1          499        Normal


In [11]:
#Analyze the spending differences between male and female customers.
query = '''
SELECT
    Gender,
    COUNT("Invoice ID") AS Total_Sales,
    ROUND(SUM("Total"), 2) AS Total_Spending,
    ROUND(AVG("Total"), 2) AS Avg_Spending_Per_Sale
FROM sales_table
GROUP BY Gender;
'''

result = pd.read_sql_query(query, conn)
print(result)

   Gender  Total_Sales  Total_Spending  Avg_Spending_Per_Sale
0  Female          501       167882.93                 335.10
1    Male          499       155083.82                 310.79


In [12]:
# Identify the most popular payment method among customers.
query = '''
SELECT
    Count("Invoice ID") AS Payment_Count,
    Payment
FROM sales_table
GROUP BY Payment
ORDER BY Payment_Count DESC
LIMIT 1;
'''

result = pd.read_sql_query(query, conn)
print(result)

   Payment_Count  Payment
0            345  Ewallet


In [13]:
#List top 5 product lines by total sales revenue.
# total sales revenue = ((number of units sold) * (the average unit price) ) + TAX
query = '''
WITH SALES AS 
(
    SELECT 
     Count("Invoice ID") AS Sold_Products,
     "Product line" AS Product_Line,
     SUM("Tax 5%") AS TAX,
      SUM(Quantity) AS Product_Quantity,
      AVG("UNIT price") AS Unit_Price
    FROM 
        Sales_table
    GROUP BY
        Product_Line
)

SELECT
    Product_Line,
    ROUND((Product_Quantity * Unit_Price)+TAX , 2) AS TOTAL_SALES_REVENUE,
    Product_Quantity AS TOTAL_QUANITY_SOLD,
    Unit_Price AS AVG_UNIT_PRICE
FROM SALES
ORDER BY TOTAL_SALES_REVENUE DESC
LIMIT 5;
'''

result = pd.read_sql_query(query, conn)
print(result)

             Product_Line  TOTAL_SALES_REVENUE  TOTAL_QUANITY_SOLD  \
0      Food and beverages             55993.99                 952   
1       Sports and travel             55058.69                 920   
2  Electronic accessories             54586.09                 971   
3     Fashion accessories             54138.59                 902   
4      Home and lifestyle             52958.58                 911   

   AVG_UNIT_PRICE  
0       56.008851  
1       56.993253  
2       53.551588  
3       57.153652  
4       55.316937  


In [14]:
#Calculate the average quantity sold per product line.
query = '''
    SELECT 
        AVG(Quantity) AS AVG_QUANTITY,
        "Product line" AS Product_Line
    FROM Sales_table
    GROUP BY Product_Line
'''
result = pd.read_sql_query(query, conn)
print(result)

   AVG_QUANTITY            Product_Line
0      5.711765  Electronic accessories
1      5.067416     Fashion accessories
2      5.471264      Food and beverages
3      5.618421       Health and beauty
4      5.693750      Home and lifestyle
5      5.542169       Sports and travel


In [15]:
#Find the product line with the highest gross margin.
query = '''
    WITH SALES AS
    (
       SELECT 
          SUM("Gross margin percentage") AS Gross_Margin,
            "Product line" AS Product_Line
        FROM Sales_table
        GROUP BY Product_Line
    )
    SELECT 
    DENSE_RANK() OVER (ORDER BY Gross_Margin DESC) SUM_Gross_Margin,
    Gross_Margin,
    Product_Line
    FROM 
    SALES
'''
result = pd.read_sql_query(query, conn)
print(result)


   SUM_Gross_Margin  Gross_Margin            Product_Line
0                 1    847.619048     Fashion accessories
1                 2    828.571429      Food and beverages
2                 3    809.523810  Electronic accessories
3                 4    790.476190       Sports and travel
4                 5    761.904762      Home and lifestyle
5                 6    723.809524       Health and beauty


In [16]:
# Discover sales trends by weekdays versus weekends.
query = '''
   SELECT
    CASE
        WHEN strftime('%w', Date) IN ('0', '6') THEN 'Weekend'
        ELSE 'Weekday'
    END AS Day_Type,
    ROUND(SUM(Total), 2) AS Total_Sales,
    COUNT("Invoice ID") AS Number_of_Transactions
FROM sales_table
GROUP BY Day_Type;
'''
result = pd.read_sql_query(query, conn)
print(result)

  Day_Type  Total_Sales  Number_of_Transactions
0  Weekday    322966.75                    1000


In [17]:
# Identify if particular product lines sell better during certain times of the day (morning vs. evening).
query = '''
  SELECT
    "Product line" AS Product_Line,
    CASE
        WHEN CAST(strftime('%H', Time) AS INTEGER) BETWEEN 6 AND 11 THEN 'Morning'
        WHEN CAST(strftime('%H', Time) AS INTEGER) BETWEEN 12 AND 17 THEN 'Afternoon'
        WHEN CAST(strftime('%H', Time) AS INTEGER) BETWEEN 18 AND 21 THEN 'Evening'
        ELSE 'Night'
    END AS Time_of_Day,
    SUM(Quantity) AS Total_Quantity_Sold
FROM sales_table
GROUP BY Product_Line, Time_of_Day
ORDER BY Product_Line, Time_of_Day;
'''
result = pd.read_sql_query(query, conn)
print(result)

              Product_Line Time_of_Day  Total_Quantity_Sold
0   Electronic accessories   Afternoon                  486
1   Electronic accessories     Evening                  298
2   Electronic accessories     Morning                  187
3      Fashion accessories   Afternoon                  490
4      Fashion accessories     Evening                  255
5      Fashion accessories     Morning                  157
6       Food and beverages   Afternoon                  441
7       Food and beverages     Evening                  332
8       Food and beverages     Morning                  179
9        Health and beauty   Afternoon                  487
10       Health and beauty     Evening                  237
11       Health and beauty     Morning                  130
12      Home and lifestyle   Afternoon                  504
13      Home and lifestyle     Evening                  194
14      Home and lifestyle     Morning                  213
15       Sports and travel   Afternoon  